# DRIVE Ground Truth Bifurcations

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scikitplot as skplt
import matplotlib.image as mpimg
import pickle5 as pickle
import glob
import csv
import os

In [5]:
# Location of the DRIVE images:
dir_images='~/retina-phenotypes/input/DRIVE/DRIVE/'
# Location of the vessels output:
dir_ARIA_output='~/retina-phenotypes/output/DRIVE/ARIA_output/'
# Location of the DRIVE Ground Truth:
dir_GT_bif ='/HDD/data/Other_datasets/DRIVE/RetinalFeatures_bif_cross/'
# Location of our DRIVE bifurcations:
dir_bif ='~/retina-phenotypes/output/DRIVE/phenotypes_all/bifurcations_position/'

In [6]:
os.chdir(dir_images) 

for img in glob.glob("*.png"):
    try:
        imageID = os.path.splitext(img)[0]
        num=imageID.split("_")[0]
        
        fig2 = plt.figure(figsize = (5,5)) # create a 5 x 5 figure 
        ax3 = fig2.add_subplot(111)
        ax3.imshow(mpimg.imread(img), interpolation='none')
        
        X = []
        Y = []
        segmentStats = []
        with open(dir_ARIA_output+ imageID + "_all_center2Coordinates.tsv") as fd:
            rd = csv.reader(fd, delimiter='\t')
            X.extend([float(j) for j in row] for row in rd)
        with open(dir_ARIA_output+ imageID + "_all_center1Coordinates.tsv") as fd:
            rd = csv.reader(fd, delimiter='\t')
            Y.extend([float(j) for j in row] for row in rd)
        with open(dir_ARIA_output+ imageID + "_all_segmentStats.tsv") as fd:
            rd = pd.read_csv(fd, sep='\t')
            segmentStats = rd["AVScore"]

        df = pd.DataFrame([])
        df["segmentStats"] = segmentStats

        df_results = pd.DataFrame([])
        df_aux = pd.DataFrame([])
        aux = int(df.count(axis=0))

        # 'Arteries' if df['AVScore'] > 0 and 'Veins' if df['AVScore'] < 0
        for i in range(aux):
            df_aux = pd.DataFrame(X[i])
            df_aux["Y"] = pd.DataFrame(Y[i])
            df_aux["type"] = segmentStats[i]
            df_aux["i"] = i
            df_results = df_results.append(df_aux, True)

        df_results.columns = ['X', 'Y', 'type', 'i']
        df_results['type'] = np.sign(df_results['type'])

        plt.scatter(x=df_results['X'], y=df_results['Y'], c=df_results['type'], cmap="jet", marker="d",
        alpha=0.5, s= 0.2)

        data = pd.read_table(dir_GT_bif + num + '_manual1_gt.txt', delimiter = ',', header=None)
        data.columns = ['X', 'Y']
        data['Row'] = data.index
        plt.scatter(data['Y'], data['X'], color='black', s=3)
        
        data_2 = pd.read_csv(dir_bif + num + '_training_bifurcations_position.csv')
        data_2['Column'] = data_2.index
        plt.scatter(data_2['Y'], data_2['X'], color='cyan', s=1.2, marker="*")
        
        from scipy.spatial.distance import cdist
        distance_matrix = cdist(data.values[:, 0:2], data_2.values[:, 0:2], 'euclidean') 
        df=pd.DataFrame(distance_matrix)

        df = df.stack().reset_index()
        df.columns = ['Row','Column','Value']

        df_filter = df[df['Value']<=7]
        data_fil=df_filter.merge(data, how='left', on='Row')
        data_2_fil=df_filter.merge(data_2, how='left', on='Column')

        #plt.scatter(data_fil['Y'], data_fil['X'], color='pink', s=0.8, marker='x')
        plt.scatter(data_2_fil['Y'], data_2_fil['X'], color='yellow', s=0.8, marker="x")
        #plt.scatter(new_df['Y'], new_df['X'], color='cyan', s=1.2)


        plt.title(f'{str(img)} N bifurcations: {len(data)} and  {len(data_2)} and  {len(data_2_fil)}')
        plt.show()
        
        print(f'{str(img)} N bifurcations: {len(data)} and  {len(data_2)} and  {len(data_2_fil)}')


    #plt.savefig(phenotype_dir + '/'+imageID + '_bif.jpg')
    #plt.close()
    except Exception as e:
        print(e)

FileNotFoundError: [Errno 2] No such file or directory: '~/retina-phenotypes/input/DRIVE/DRIVE/'